In [ ]:
# Estimator 란
# tf.estiamtor 는 Python의 대표적인 머신러닝 패키지인 scikit-learn(sklearn) 스타일처럼 복잡한 딥러닝 모델을 쉽게 작성할 수 있도록 해주는 라이브러리이다.
# keras와 마찬가지로 if.contrib.learn으로 텐서플로우에 들어왔으며, 1.11.0 텐서플로우 버전부터  tf.estimator로 옮겨졌다.

In [ ]:
# 2. Pre-made vs. Custom
# pre-made estimator들은 tf.estimatore.Estimator 기반 클래스의 하위 클래스sub class이며, custom Estimator는 
# tf.estimator.Estimator의 instance이다.
# pre-made Estimator 들은 DNNClassifier 나 LinearRegressor 처럼 모델이 미리 구현되어 있어, 사용자가 빠르게 이를 이용할 수 있다.
# 그러나, CNN 또는 RNN 등과 같은 모델은 미리 구현되어 있지 않기 때문에, 텐서플로의 Estimator를 이용하여 이러한 모델들을 학습시키기 위해서는 
# custom Estimator로 사용자가 직접 Estimator를 만들어 줘야 한다.

In [ ]:
# 2.1  model function
# Estimator에서 model function 또는 model_fn 이 바로 ML/DL 알고리즘이 수행되는 함수이다. 즉 pre-made Estimator와 custom Estimator의 차이점은 다음과 같다
#  1. pre-made Estimator는 model_fn이 미리 구현되어 있어, 별도의 프로그래밍 없이 바로 train()/evaluate()/predict()를 사용할 수 있다.
#  2. 반면 custom Estimator는 model_fn 이 구혀노디어 있지 않으므로, 반드시 model_fn에 사용자가 직접 알고리즘을 구현해 줘야 한다.

# model function에는 모든 종류의 hidden layer와 metric을 가지고 다양한 ML/DL 알고리즘을 모델링 할 수 있으며, 텐서플로 Layers와 Metrics API를 이용해 구현한다


In [ ]:
# 3. Custom Estimator 분류기 구현하기
# 위에서 살펴본 내용을 토대로 붓꽃 데이터 iris data를 분류하는 classifier를 단계별로 구현해 보자. 구현할 모델은 2개의 hidden layer를 가지며,
# 각 hidden layer마다 10개의 노드로 구성되어 있다.
# iris dataset은 http://download.tensorflow.org/data/iris_training.csv, http://download.tensorflow.org/data/iris_test.csv 파일을 다운받아 처리한다.


In [5]:
import pandas as pd
import tensorflow as tf

TRAIN_FILE='./data/iris_training.csv'
TEST_FILE='./data/iris_test.csv'

CSV_COLUMN_NAMES=['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES=['Setosa', 'Versicolor', 'Virginica']

def load_data(y_name='Species'):
    train = pd.read_csv(TRAIN_FILE, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)
    
    test = pd.read_csv(TEST_FILE, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y =  test, test.pop(y_name)
    
    # run in jupyter notebook
    print('Train set')
    display(train.head(5))
    print('Test set')
    display(test.head(5))
    
    return (train_x, train_y), (test_x, test_y)

In [6]:
(train_x, train_y), (test_x, test_y) = load_data()

Train set


,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


Test set


,SepalLength,SepalWidth,PetalLength,PetalWidth
0,5.9,3.0,4.2,1.5
1,6.9,3.1,5.4,2.1
2,5.1,3.3,1.7,0.5
3,6.0,3.4,4.5,1.6
4,5.5,2.5,4.0,1.3


In [ ]:
# 3.1.2 Input function
# 구현한 Estimator에 training, evaluating, predicting 을 위한 데이터를 제공하기 위해서는 input_function을 반드시 구현해야 한다. 

In [12]:
feature_names = [
    'SepalLength',
    'SepalWidth',
    'PetalLength',
    'PetalWidth'
]

In [ ]:
# input_fn()의 구성 모습 형식...
# def input_fn():
#     ... <code> ...
#     return ({'SepalLength':[values], ... , 'PetalWidth':[values]}, [IrisFlowerType])
#
# The return value must be a two-element tuple organized as follows:
#  1) The first element must be a dict in which each input feature is a key, and then a list of values for the training batch
#  2) The second element is a list(1-d tensorflow tensor) of labels for the training batch

# input_fn을 재사용하기 위해서 매개변수를 추가한다. 
#   file_path : the data file to read
#   perform_shuffle : whether the record order should be randomized.
#   repeat_count : The number of times to iterate over the records in the dataset. If we specify 1, then each record is read once.
#                  If we specify None, iteration will continue forever.

In [14]:
def my_input_function(file_path, perform_shuffle=False, repeat_count=1, batch_size=32):
    # file_path : 읽어들일 파일 경로
    # perform_shuffle : 데이터순서를 무작위로 섞을지 여부를 지정
    # repeat_count : 데이터 셋을 반복할 회수.. 즉 epoch수에 해당한다고 볼 수 있다.
    #   repeat_count 가 1이면 각 데이터셋 데이터를 1번만 읽어들인다.
    #   repeat_count가 None 으로 지정되면, 계속적으로 데이터셋을 반복한다.
    def decode_csv(line):
        parsed_line = tf.decode_csv(line, [[0.], [0.], [0.], [0.],[0.]])
        label = parsed_line[-1] # Last element is the label
        del parsed_line[-1] # delete last element
        features = parsed_line # everything but last element are the features
        d = dict(zip(feature_names, features)), label
        return d
    
    dataset = (tf.data.TextLineDataset(file_path) # Read text file
            .skip(1) # skip header row
            .map(decode_csv, num_parallel_calls=4) # Decode each line
#             .cache() # Warning : Caches entire dataset, can cause out of memory
#             .shuffle(shuffle_count)  # Randomize elems (1 == no operation)
#             .repeat(repeat_count) # Repeats dataset this # times
#             .batch(batch_size)
#             .prefetch(1) # Make sure you always have 1 batch  ready to serve
    )
    if perform_shuffle:
        # Randomize input using a window of 256 elemnts (read into memory)
        dataset = dataset.shuffle(buffer_size=256)
    
    
    dataset = dataset.repeat(repeat_count) # Repeats dataset this # times
    dataset = dataset.batch(batch_size)  # batch size to use
    
    
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels


In [ ]:
# input_fn은 다음 2개의 값을 반환한다.
# 1. batch_features : Dictionary 사전 데이터입니다. Dictionary Key는 feature 이름이고, 사전의 값 valule는 feature 값이다.
# 2. batch_labels : 배치 레이블 값 list 입니다.

In [ ]:
# TextLineDataset : The Dataset API will do a lot of memory management for you when you're using its file-based datasets. 
#    You can, for example, read in dataset files much larger than memroy or read in multiple files by specifying a list as a argument.
# shuffle : Read buffer_size records, then shuffles (randomized) their order.
# map : calls the decode_csv funtion with each element in the dataset as an argument (since we are using TextLineDataset, each element
#       will be a line of CSV text). Then we apply decode_csv to each of lines.
# decode_csv : splits each line into fields, providing the default values if necessary. Then returns a dict with the field keys and field 
#              values. The map function updates each elem (line) in the dataset with the dict.

In [16]:
next_batch = my_input_function(TRAIN_FILE, True) # will return default batch size 32 random elements

# Now let's try it out, retrieving and printing one batch of data. 
# Although this code looks strange, you don't need to understand
# the details.

with tf.Session() as sess:
    first_batch = sess.run(next_batch)
    
print(first_batch)


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
({'SepalLength': array([6.5, 7.7, 4.7, 4.9, 5.1, 4.6, 7.7, 7.9, 5. , 5.8, 5.8, 5.7, 6.4,
       6.8, 7.7, 6.3, 5.1, 6.7, 6.2, 4.6, 6.9, 5.7, 6.1, 6.8, 5. , 5.5,
       4.6, 4.6, 6.7, 6.3, 6.9, 5.8], dtype=float32), 'SepalWidth': array([3. , 2.8, 3.2, 3.1, 3.8, 3.2, 3.8, 3.8, 3.5, 2.7, 2.7, 2.8, 3.2,
       3. , 3. , 2.5, 3.5, 3.1, 2.8, 3.1, 3.1, 3.8, 2.8, 2.8, 3.5, 2.6,
       3.4, 3.6, 3.1, 3.4, 3.2, 2.7], dtype=float32), 'PetalLength': array([5.8, 6.7, 1.3, 1.5, 1.5, 1.4, 6.7, 6.4, 1.3, 5.1, 4.1, 4.1, 5.3,
       5.5, 6.1, 5. , 1.4, 4.4, 4.8, 1.5, 5.1, 1.7, 4.7, 4.8, 1.6, 4.4,
       1.4, 1. , 5.6, 5.6, 5.7, 5.1], dtype=float32), 'PetalWidth': array([2.2, 2. , 0.2, 0.1, 0.3, 0.2, 2.2, 2. , 0.3, 1.9, 1. , 1.3, 2.3,
       2.1, 2.3, 1.9, 

In [ ]:
# Create feature columns 특징 열 생성
# 모델의 feature column을 정의하여 각 feature의 표현을 지정해야 한다. pre-made Estimator 를 사용하든 custom Estimator를 사용하든 동일한 방식으로 feature column을
# 정의한다. 

In [17]:
feature_columns = [
    tf.feature_column.numeric_column(feature_names[0]),
    tf.feature_column.numeric_column(feature_names[1]),
    tf.feature_column.numeric_column(feature_names[2]),
    tf.feature_column.numeric_column(feature_names[3])
]

In [ ]:
# 모델 함수 작성 write a model function
# 이제 커스텀 Estimator용 model_fn을 작성할 준비가 되었습니다. 함수 선언부터 시작해 봅시다.
#
# def my_model_fn(
#   features, # This is batch_features from input_fn
#   labels,  # this is batch_labels from input_fn
#   mode) :  # instance of tf.estimatore.ModeKeys, see below....

In [ ]:
# model_fn의 두개의 인자는 input_fn에서 반환되는 features와 labels 이다. 즉 features와 labels는 모델에서 사용할 데이터에 대한 핸들이다.
# mode 인자는 호출자가 훈련, 예측 또는 평가를 요청하는지 여부를 가리킨다...
# 일반적인 model_fn을 구현하기 위해서는 다음을 수행해야 한다.
#  1. 모델 계층을 정의해야 한다.
#  2. 세가지 다른 모드에서 모델의 행동을 규정해야 한다.

In [ ]:
# 모델 계층 정의 Define the model's layers
# 사용자 Estimator가 심층 신경망을 생성한다면, 다음 세가지 레이어를 정의해야 한다.
#  1) 입력 레이어 an input layer
#  2) 하나이상의 히든 레이어  one or more hidden layers
#  3) 출력 레이어  an output layer
# 은닉층들과 출력 층을 정의하기 위해 tf.layers라는 layer API를 사용한다.
# 만약 사용자 정의 Estimator가 선형 모델을 생성한다면, 다음에 설명하듯이 하나의 레이어만 생성한다.

In [ ]:
# 입력 계층 정의 Define the input layer
# tf.feature_column.input_layer를 호출하여 심층 신경망의 입력 레이어를 정의한다.
#
# Create the layer of input
# input_layer = tf.feature_column.input_layer(features, feature_columns)
#

# 위코드는 입력계층을 생성하고, 입력함수를 통해 features를 읽어내고, 이전에 정의한 feature_columns  를 통해 필터링한다.
# 선형 모델의 입력층을 생성하기 위해서, tf.feature_colum.input_layer 대신에 tf.feature_column.linear_model을 호출한다.
# 선영 모델은 은닉층이 없으므로, tf.feature_column.linear_model에서 반환되는 값은 입력 레이어 또는 출력 레이어 양쪽을 모두 수행한다.
# 다른 얘기로, 이 함수의 반환 값은 예측이다. 

In [ ]:
# # 은닉 계층 구성 establish hidden layers
# # 심층신경망을 생성한다면, 반드시 하나 이상의 은닉계층을 정의해야 한다. Layers API는 컨볼루션, 풀링, 그리고 드롭아웃 계층을 포함한 모든 유형의 은닉 계츧을을 정의하는 풍부한 함수 집합을 제공한다. 
# # Iris 데이터의 경우, tf.layers.Dense를 두번 호출하여 두개의 조밀한 은닉 층을 생성한다. 각 은닉층은 10개의 뉴런을 가지고 있다. "dense"의 의미는 첫번째 은닉 층의 각각의 뉴런은 두번째 은닉층의 모든 뉴런\
# # 들과 연결된다는 것응 릐미한다.
# # 다음은 관련된 코드이다.
# # 
# # Definition of hidden layer : h1
# # (Dense returns a Callable so we can provide input_layer as argument to it)
# h1 = tf.layers.Dense(10, activation=tf.nn.relu)(input_layer)

# # Definition of hidden layer : h2
# # (Dense returns a Callable so we can provide h1 as argument to it)
# h2 = tf.layers.Dens(10, activation=tf.nn.relu)(h1)

# # tf.layers.Dense 의 inputs 파라메터는 선행 레이어를 식별한다. h1 레이어의 선행 레이어는 input_layer 이다.
# # 또한 h2의 선행 레이어는 h1 이다. 
# #
# # tf.layers.Dense의 첫번째 인자는 출력 뉴런 갯수를 정의한다. 이경우에는 10 이다. 
# # activation 인자는 활성화 함수를 정의한다. 이 경우에는 Relu 이다. 

# # tf.layers.Dense 는 다양한 정규화 매개변수를 설정하는 기능을 포함한 많은 부가적인 기능을 제공한다. 간단히 학 위해
# # 다른 파라메터들은 기본값을 사용하는 것으로 했다. 
# # 또한 tf.layers 를 살펴보면, tf.layers.dense 라는 솟문자 버전을 볼 수 있을 것이다. 일반적으로 대문자로 시작하는 클래스 버전을 
# # 사용해야 한다. (tf.layers.Dense)

In [ ]:
# 출력계층 Output layer
# tf.layers.Dense를 다시 호출하여 출력 레이어를 정의할 것이다.
# # Output 'logits' layer is three members = probability distribution
# # (Dense returns a Callable so we can provide h2 as argument to ti)
# logits = tf.layers.Dense(3)(h2)

In [ ]:
# 출력 계층은 h2로 부터 입력을 받아들이는 것을 유의하라.
# 출력 레이어를 정의할 때, units 파라메터는 가능항 출력 값들의 갯수를 정의한다.  따라서  units을 3 으로 설정함으로써, tf.layers.Dense 함수는 3 요소 로짓 벡터를 구성한다.  로짓 벡터의 각 셀은 iris 데이터가 각각 Setosa, Versicolor 또는 Virginica 3 종류중
# 어디에 속하는 지에 대한 확률이 포함되어 있습니다.
#
# 출력 레이어가 최종 레이어이기 때문에, tf.layers.Dense 를 호출할 때 선택사항인 activation 파라메터는 생략한다.

In [ ]:
# Implement training, evluation, and prediction 훈련, 평가 및 예측 구현
# model_fn  생성의 마지막 단계는 예측, 평가, 그리고 훈련을 구현하는 분기코드를 작성하는 것이다.
# model_fn 은 누군가 Estimator의 train, evaluate, predict 함수를 호출할 때마다 호출된다. model_fn의 함수 서명은 다음과 같은 형태입니다.
#
# def my_model_fn(
#     features, # This is batch_features from input_fn
#     labels,  # This is batch_labels from input_fn
#     mode) :  # instance of tf.estimator.ModeKeys, see blow

In [ ]:
# 세번째 인자인 mode에 주목하자. 아래의 테이블에서 보여주듯이, 누군가 train, evaluate, 그리고 predict ㅁ함수를 호출했을 때, Estimator 프레임웤는 model 인자가 다음과
# 같이 설정된 모델 함수를 호출합니다.
#
#         Estimator 함수 호출                Estimator 프레임워크는 mode 인자를 다음값으로 설정
#           train()                          ModeKeys.TRAIN
#           evaluate()                       ModeKeys.EVAL
#           predict()                        ModeKeys.PREDICT

In [ ]:
# 예를들어, classifier라고 불리는 사용자 정의 Estimator를 인스턴스화 한다고 가정하자.
# 다음과 같은 호출을 수행한다. 
#  classifier.train(
#      input_fn = lambda: my_input_fn(FILE_TRAIN, repeat_count=500, shuffle_count=256))
#

In [ ]:
# 그러면 Estimator 프레임워크는 ModeKeys.TRAIN으로 설정된 model 인자를 가지고 model_fn을 호출한다.
#
# model_fn 은 반드시 mode 값이 가지는 세가지 경우를 핸들링하는 코드를 제공한다.
# 각각의 mode 값에 대하여, model_fn은 반드시 tf.estimator.EstimatorSpec을 반환해야 한다. EstimatorSpec은 호출자가 요구한 정보를 포함하고
# 있다. 각각의 모드를 검토해 보자..

In [ ]:
# PREDICT
# model_fn 이  mode=ModeKeys.PREDICT로 호출되었을 때, 모델 함수는 반드시 다음의 정보들을 포함하는 tf.estimator.EstimatorSpec을 반환해야 한다.
#  . the mode, tf.estimator.ModeKeys.PREDICT
#  . the prediction
# 모델은 반드시 예측을 하기 전에 학습되어야 한다. 훈련된 모델은 Estimator를 생성시킬 때 설정된 디렉토리 model_dir에  저장된다.
# 이번 경우, 예측을 생성하는 코드는 다음과 같다.

In [ ]:
# # class_ids will be the model prediction for the class (Iris flower type) 
# # The output node with the highest vlaue is our prediction
# predictions = {'class_ids':tf.argmax(input=logits, axis=1)}

# # return our prediction
# if mode == tf.estimator.ModeKeys.PREDICT:
#     return tf.estimator.EStimatorSpec(mode, predictions=predictions)

In [ ]:
# prediciton block이 놀랍게 짧습니다. 이 코드들은 단지 굴러오는 예측을 잡는 긴 호스 끝의 통일 뿐입니다. 결국, Estimator는 예측을 수행하기 위한 어려운 과정을 이미 모두 완료했습니다.
# 1. 입력함수 input_fn는 모델함수model_fn에 추론할 데이터(feature values)를 제공한다.
# 2. 모델 함수 model_fn는 그러한 feature values를 feature_columns로 변환한다.
# 3. 모델 함수는 사전에 학습된 모델을 통해 해당 feature column을 실행한다.

In [ ]:
# 출력층은입력 된 꽃이 3가지 붓꽃 종류 각각의 값을 포함하고 있는 logits 벡터이다. 
# tf.argmax 메소드는 logits 벡터에서 가장 큰 값을 가지고있는 iris 종을 선택한다.

# 가장 높은 값은 'class_ids' 라는 사전 키에 값으로 할당된다. tf.estimator.EstimatorSpec 의 predictions 매개변수를 통해서 해당 사전을 반환한다.
# 호출자는 Estimator의 predict 메소드의 반환값으로 전달된 사전을 검사하여 prediction을 추출할 수 있다.

In [ ]:
# EVAL
# model_fn 이 mode == ModeKeys.EVAL 로 호출되었을 때, 모델 함수는 모델을 평가해야 하고, 손실과 한가지 이상의 측정 값을 반환해야 한다.
# 현재 예에서는 tf.losses.sparse_softmax_cross_entropy 를 호출함으로써 손실을 계산할 수 있다.
# 다음은 완전한 코드이다.

In [ ]:
# # To calculate the loss, we need to convert our labels
# # Our input labels have shape: [batch_size, 1]
# labels = tf.squeeze(labels, 1)    # convert to shape [batch_size]
# loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

In [ ]:
# 이제 측정값에 주목해보자.. 비록 측정값을 반환하는 것은 선택사항이지만, 대부분의 사용자 정의 Estimator들은 적어도 하나의 측정값을 반환한다.
# TensorFlow는 다른 종류의 측정값을 계산하기 위한 Metrics API (tf.metrics)를 제공한다. 간략하기 위해, 단지 정확도accuracy만 반환한다. 
# tf.metrics.accuracy는 실제 정답(즉 input_fn에 의해 제공된 labels)과 모델의 예측값을 비교한다.
# tf.metrics.accuracy 함수는 같은 모양 same shape을 가지는 labels와 예측값 predictions를 요구한다. 아래에 tf.metrics.accuracy를 호출하는 예이다.

In [ ]:
# Calculatre the accuracy between the true labels, and our predictions
accuracy = tf.metrics.accuracy(labels, predictions['class_id'])

In [ ]:
# 모델이 mode==ModeKeys.EVAL 값을 가지고 호출되었다면, 모델 함수 model_fn은 다음의 정보를 포함하는 tf.estimator.EsticmatorSpec 을 반환한다.
# . the mode, 이 값은 tf.estimator.ModeKeys.EVAL
# . the model's loss
# . 전형적으로, 하나이상의 측정값으로 사전에 포함된다.
#
# 그래서 여기서는 단 하나의 측정값(my_accuracy)을 포함하는 사전을 생성한다. 만약 다른 측정값을 계산했다면, 우리는 동일한 사전에 부가적인 키/값 쌍을 추가할 것이다.
# 그런다음, tf.estimator.EstimatorSpec의 eval_metric_ops 매개변수에 그 사전을 넘겨줄 것이다. 
# 아래는 해당 코드 블럭이다.

In [ ]:
# Return out loss (which is used to evaluate our model)
# Set the TensorBoard scalar my_accuracy to the accuracy
# Obs : This function only sets value during mode==ModeKeys.EVAL
# To set values during training, see tf.summary.scalar
if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(
        mode,
        loss=loss,
        eval_metric_ops={'my_accuracy':accuracy})

In [ ]:
# TRAIN
# mode == ModeKeys.TRAIN 으로 모델함수 model_fn이 호출되었을 때, 모델 함수 model_fn은 모델을 학습시켜야 한다.
# 첫째로 최적화 객체 optimizer object를 인스턴스화 한다(instantiate 생성해야 한다. )
# 이번 예제에서는 Adagrad를 사용하는 DNNClassifier를 모방하기 위해서, 다음의 코드 블록에서 Adagrad 최적화기를 선택한다
#  tf.train.AdagradOptimizer
# tf.trains 패키지는 많은 다른 최적화기를 제공한다. 자유롭게 실험해 보시라...

In [ ]:
# 다음으로 옵티마이저에 관한 목적을 설정한다. 옵티마이저의 목적은 단순히 loss를 최소화 하는 것이다. 
# 그러한 목적을 세우기 위해, 우리는 minimize 메소드를 호출할 것이다. 
# 아래 코드에서, 선택사항값인 global_step 인자는 TensorFlow가 처리된 배치숫자를 계산하기 위해 사용되는 변수를 지정한다.
# global_steps를 tf.train.get_global_step에 설정하는 것은 멋지게 작옹할 것이다. 
# 또한 우리는 학습하는 동안에 TensorBoard에 my_accuracy를 보고하기 위해서 tf.summary.scalar를 호출할 것이다.
# 이 과정에 대한것은 아래 TensorBoard에 관한 절을 참조하라..

In [ ]:
# optimizer = tf.train.AdagradOptimizer(0.05)
# train_op = optimizer.minimize(
#     loss,
#     global_step = tf.train.get_global_step())

# # Set the TensorBoard scalar my_accuracy to the accuracy
# tf.summary.scalar('my_accuracy', accuracy[1])

In [ ]:
# mode == ModeKeys.TRAIN 값을 가지고 모델을 호출할 때, 모델 함수 model_fn 은 반드시 다음의 정보들을 포함하는 tf.estimator.EstimatorSpec을 반환해야 한다.
#  . the mode, 이 값은 tf.estimator.ModeKeys.TRAIN 이다.
#  . loss
#  . the result of the training op

# 코드는 다음과 같다..


In [ ]:
# # Return training operations: loss and train_op
# return tf.estimator.EstimatorSepc(
#     mode,
#     loss = loss,
#     train_op = train_op)

In [ ]:
# now our model_fn is complete!

In [ ]:
#사용자 정의 Estimator The custom Estimator

In [ ]:
# 새로운 사용자 정의 Estimator를 생성한 후, 사용해 보고 싶을 것이다. Esatimator 기본 클래스를 통해서, 사용자 정의 Estimator를 인스턴스화부터 시작한다.

In [ ]:
# classifier = tf.estimator.Estimator(
#     model_fn = my_model_fn,
#     model_dir =PATH) #Path to where checkpoints etc are stored.

In [ ]:
# 우리의 Estimator를 train, evaluate, and predict하는 나머지 코드는 제 1붕에서 설명한 사전제작된 DNNClassifier  경우와 동일하다.
# 예를 들어,. 모델을 학습시키기 위해 다음과 같이 호출할 수 있다.

In [ ]:
# classifier.train(
#     input_fn = lambda : my_input_fn(FILE_TRAIN, repeat_count=500, shuffle_count=256))